In [1]:
from train_data_processor import TrainDataProcessor
# from test_data_processor import TestDataProcessor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_file_path = "data/train.csv"
train_data_processor = TrainDataProcessor(train_file_path)
train_df,columns_to_dropped = train_data_processor.run_process()

In [3]:
train_df.drop(columns=columns_to_dropped, inplace=True)
columns_to_drop = ["Universite Adi","Universite Kategori",'Lise Adi','Lise Sehir']
train_df.drop(columns=columns_to_drop, inplace=True)

In [4]:
from sklearn.impute import KNNImputer

# KNN Imputer kullanarak NaN değerlerini doldurma
imputer = KNNImputer(n_neighbors=5)
train_df = pd.DataFrame(imputer.fit_transform(train_df), columns=train_df.columns)

In [5]:
# Basvuru Yili 2016 olan satırları düşürme
# train_df = train_df[train_df['Basvuru Yili'] != 2016]
# train_df["Basvuru Yili"].unique()
## 2016 olan değerleri yarı yarıya azaltıyorum, daha smooth bir hale getirmek için.
train_df.loc[train_df['Basvuru Yili'] == 2016, 'Degerlendirme Puani'] = train_df.loc[train_df['Basvuru Yili'] == 2016, 'Degerlendirme Puani'] / 2
train_df['Basvuru Yili'] = train_df['Basvuru Yili'] - 2000

In [6]:
# columns_to_convert = train_df.drop('Degerlendirme Puani', axis=1).columns
# # Kategorik sütunları integer (int) türüne çevirme
# # Eğer sütunlarda kategorik veri varsa, bu verileri int türüne çevirmeden önce uygun dönüşüm yapılmalıdır
# train_df[columns_to_convert] = train_df[columns_to_convert].apply(lambda col: pd.to_numeric(col, errors='coerce').astype('Int64') if col.dtype == 'object' else col)

In [7]:
# # 'anne' kelimesini içeren sütunları almak
# def stack_anne_baba_egitim_column(df,filter):
#     # egitim_columns = df.filter(like=filter).columns

#     df[f'{filter}_Egitimsiz'] = df[f'{filter}_İlkokul'] + df[f'{filter}_Ortaokul'] + df[f'{filter}_Eğitimi yok']
#     df[f'{filter}_Universite'] = df[f'{filter}_Üniversite'] + df[f'{filter}_Yüksek Lisans'] + df[f'{filter}_Doktora']

#     df.drop(columns=[f"{filter}_İlkokul",f"{filter}_Ortaokul",f"{filter}_Eğitimi yok",f"{filter}_Üniversite",f"{filter}_Yüksek Lisans",f"{filter}_Doktora"],axis=1,inplace=True)
#     return df

# train_df = stack_anne_baba_egitim_column(train_df,"anne")
# train_df = stack_anne_baba_egitim_column(train_df,"baba")

In [8]:
train_df.shape

(46973, 71)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Veri setini özellikler (X) ve hedef değişken (y) olarak ayırma
X = train_df.drop('Degerlendirme Puani', axis=1)
y = train_df['Degerlendirme Puani']
# Eğitim ve test veri setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor

# Hiperparametre aralıklarını belirleme
param_distributions = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'iterations': [500, 1000],
    'l2_leaf_reg': [3,5, 7, 9]
}

# Model oluşturma
model = CatBoostRegressor(
    random_seed=42,
    eval_metric='RMSE'
)

# RandomizedSearchCV uygulama
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=20,  # Denenecek hiperparametre kombinasyonu sayısı
    scoring='neg_mean_squared_error',
    cv=5,  # 5-fold cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

# En iyi parametreler
print(f'En iyi hiperparametreler: {random_search.best_params_}')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0:	learn: 13.6447951	total: 142ms	remaining: 2m 21s
1:	learn: 12.6991722	total: 148ms	remaining: 1m 13s
2:	learn: 11.8843596	total: 155ms	remaining: 51.6s
3:	learn: 11.1688399	total: 161ms	remaining: 40.1s
4:	learn: 10.5477486	total: 167ms	remaining: 33.2s
5:	learn: 10.0300676	total: 175ms	remaining: 28.9s
6:	learn: 9.5498811	total: 180ms	remaining: 25.6s
7:	learn: 9.1563169	total: 188ms	remaining: 23.3s
8:	learn: 8.7968850	total: 194ms	remaining: 21.4s
9:	learn: 8.4919592	total: 201ms	remaining: 19.9s
10:	learn: 8.2415521	total: 249ms	remaining: 22.4s
11:	learn: 8.0110304	total: 292ms	remaining: 24.1s
12:	learn: 7.8278325	total: 329ms	remaining: 25s
13:	learn: 7.6700725	total: 369ms	remaining: 26s
14:	learn: 7.5230632	total: 432ms	remaining: 28.4s
15:	learn: 7.3923791	total: 473ms	remaining: 29.1s
16:	learn: 7.2827693	total: 512ms	remaining: 29.6s
17:	learn: 7.1903862	total: 567ms	remaining: 30.9s
18:	learn: 7.0870522	total

In [11]:
# En iyi model ile tahmin
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

In [12]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Eğitim hatası
train_pred = best_model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))

# Test hatası
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'Eğitim RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')

# Overfitting veya underfitting kontrolü
if train_rmse < test_rmse:
    print("Overfitting olabilir.")
elif train_rmse == test_rmse:
    print("Model dengeli.")
else:
    print("Underfitting olabilir.")

Eğitim RMSE: 4.792001122787139
Test RMSE: 5.487089097931404
Overfitting olabilir.


In [121]:
# KNN yöntemi ile RMSE: 5.481886156724653
# KNN yöntemi ve basvuru yılından 2000 çıkartınca RMSE: 5.479630834403727
# universiteleri sınıflandırmadan direkt puanıyla eğitince RMSE: 5.4786335267308095
# Anne ve baba sektor eklemesi yapıldıktan sonra RMSE: 5.467997028898569
# pc ve elektronik mühendisliklerini ekledikten sonra RMSE: 5.448576823848511

In [12]:
# Feature önem derecelerini elde etme
feature_importances = model.get_feature_importance()

# Özellik isimlerini al
feature_names = X.columns

# Feature importances DataFrame oluşturma
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)
# Özelliklerin önem derecelerini görselleştirme
# plt.figure(figsize=(10, 8))
# plt.barh(importance_df['Feature'], importance_df['Importance'])
# plt.xlabel('Importance')
# plt.ylabel('Feature')
# plt.title('Feature Importances')
# plt.gca().invert_yaxis()
# plt.show()

In [ ]:
importance_df.head(20)

In [126]:
# Basvuru Yilini dt yapınca -> RMSE: 5.883411328590871
# int iken -> RMSE: 5.888681272566932
# 18<= Age <= 30 filtresinden sonra  RMSE: 5.917980386760992
# 18<= Age <= 25 filtresinden sonra  RMSE: 5.848790912079283